In [1]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Assign omegas using sin()/cos() concept
C2v symmetry is required
KKI 6/5/2023
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)

In [2]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'
fsoci = '../UMemphis/mgcl-equil-karl-block.pro'
#fsoci = '../UMemphis/no.pro'
#fsoci = '../UMemphis/mgcl_small4.pro'
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ../UMemphis/mgcl-equil-karl-block.pro


In [3]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 2.3000
22 Doublets
Active space = 9/8
Uniform weighting


In [4]:
# count the terms that are included in the calculation
from collections import Counter
for spin, grp in CAS.results.groupby('Spin'):
    print(spin)
    print(Counter(grp.Term.tolist()))
#CAS.results

Doublet
Counter({'2Π': 10, '2Σ+': 6, '2Δ': 4, '2Σ-': 2})


In [5]:
casdf = CAS.results[['Irrep', 'Label', 'Energy', 'Term']].copy()
casdf['S'] = [chem.spinname(x)-1 for x in CAS.results.Spin]
casdf['Lz'] = np.round(np.sqrt(np.abs(CAS.results.LzLz)), 0).astype(int)
spin = sorted(set(casdf.S))
irreps = sorted(set(casdf.Irrep))
lzvals = sorted(set(casdf.Lz))

In [6]:
def assign_trig(Lzs, irreps):
    # Add the sin/cos descriptor based upon Lz and irrep
    trig = []
    for L, irr in zip(Lzs, irreps):
        if L == 0:
            trig.append('1')
        elif L%2 == 1:
            # odd L
            if irr == 2:
                trig.append('cos')
            elif irr == 3:
                trig.append('sin')
            else:
                trig.append('???')
        else:
            # even L
            if irr == 1:
                trig.append('cos')
            elif irr == 4:
                trig.append('sin')
            else:
                trig.append('???')
    return trig

In [7]:
casdf['ecm'] = np.round((casdf.Energy - casdf.Energy.min()) * chem.AU2CM, 0)
print('CASSCF results')
casdf.sort_values('Energy')

CASSCF results


,Irrep,Label,Energy,Term,S,Lz,ecm
0,1,1.1,-660.906685,2Σ+,1,0,0.0
8,2,1.2,-660.787376,2Π,1,1,26185.0
13,3,1.3,-660.787376,2Π,1,1,26185.0
9,2,2.2,-660.775209,2Π,1,1,28855.0
14,3,2.3,-660.775209,2Π,1,1,28855.0
1,1,2.1,-660.748771,2Σ+,1,0,34658.0
2,1,3.1,-660.730059,2Σ+,1,0,38765.0
3,1,4.1,-660.718892,2Δ,1,2,41216.0
18,4,1.4,-660.718892,2Δ,1,2,41216.0
19,4,2.4,-660.716449,2Σ-,1,0,41752.0


In [8]:
# check for proper pairing of degenerate states
broken = False
for grp, dfg in casdf.groupby(['S', 'Lz']):
    if grp[1] == 0:
        # ignore Sigma states
        continue
    # number in each irrep should be equal
    grpi = dfg.groupby('Irrep')
    lens = grpi.size().values
    if lens[0] != lens[1]:
        print('Broken pair somewhere')
        display(dfg.sort_values('Energy'))
        broken = True
if not broken:
    print('All CASSCF state pairs are closed')

All CASSCF state pairs are closed


In [9]:
# check for non-integer Lz values
cruft = casdf.Lz - np.round(casdf.Lz, 0)
if cruft.any():
    print('*** There are non-integer values of Lz')
else:
    print('Lz values look OK')

Lz values look OK


In [10]:
# order the CASSCF states by energy within each (S, irrep) group
newdf = pd.DataFrame(columns=casdf.columns)
for lbl, dg in casdf.groupby(['S', 'Irrep']):
    dt = dg.copy().sort_values('Energy').reset_index(drop=True)
    #display(dt)
    newdf = newdf.append(dt)

In [11]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist)
# Add the trig column
dfci['trig'] = assign_trig(dfci.Lz, dfci.Irrep)

In [12]:
dfci['ecm'] = np.round( (dfci.Edav - dfci.Edav.min()) * chem.AU2CM, 1)
print('MRCI results')
dfci.sort_values('Edav')

MRCI results


,Group,Spin,Irrep,Label,Energy,Edav,Ncore,dipX,dipY,dipZ,Eref,Dipole,Ref,C0,Configs,Lz,Term,trig,ecm
0,1,Doublet,1,1.1,-661.161910,-661.189203,10,0.0,0.0,-1.564913,-660.906685,1.564913,1.1,0.950992,"{'22a02020': 0.9269158, '220a2020': -0.1195967...",0.0,2Σ+,1,0.0
8,3,Doublet,2,1.2,-661.037247,-661.069028,10,0.0,0.0,-1.980664,-660.787376,1.980664,1.2,0.942952,"{'22aab020': -0.0199338, '22aba020': 0.0384188...",1.0,2Π,cos,26375.4
13,4,Doublet,3,1.3,-661.037247,-661.069028,10,0.0,0.0,-1.980664,-660.787376,1.980664,1.3,0.942952,"{'22aa20b0': -0.0199338, '22ab20a0': 0.0384188...",1.0,2Π,sin,26375.4
14,4,Doublet,3,2.3,-661.003948,-661.025732,10,0.0,0.0,1.420276,-660.775209,1.420276,2.3,0.955114,"{'22aa20b0': 0.0206926, '22ab20a0': 0.1642669,...",1.0,2Π,sin,35877.8
9,3,Doublet,2,2.2,-661.003948,-661.025732,10,0.0,0.0,1.420276,-660.775209,1.420276,2.2,0.955114,"{'22aab020': 0.0206926, '22aba020': 0.1642669,...",1.0,2Π,cos,35877.8
1,1,Doublet,1,2.1,-660.987386,-661.019217,10,0.0,0.0,-1.960257,-660.748771,1.960257,2.1,0.941384,"{'22a02020': 0.0645692, '220a2020': 0.8081035,...",0.0,2Σ+,1,37307.5
2,1,Doublet,1,3.1,-660.968666,-660.993563,10,0.0,0.0,0.760857,-660.730059,0.760857,3.1,0.950590,"{'22a02020': -0.090828, '220a2020': -0.3384821...",0.0,2Σ+,1,42938.0
6,2,Doublet,1,4.1,-660.934526,-660.952830,10,0.0,0.0,1.382673,-660.718892,1.382673,4.1,0.960083,"{'22a020ba': 0.585081, '22a0ba20': -0.5850799,...",2.0,2Δ,cos,51877.9
18,5,Doublet,4,1.4,-660.934526,-660.952829,10,0.0,0.0,1.382691,-660.718892,1.382691,1.4,0.960084,"{'22a02ab0': 0.6757897, '22a02ba0': -0.0003395...",2.0,2Δ,sin,51878.0
20,6,Doublet,4,2.4,-660.932190,-660.950473,10,0.0,0.0,1.394746,-660.716449,1.394746,2.4,0.960145,"{'22a02ab0': 0.67567, '22a02ba0': -0.0144656, ...",0.0,2Σ-,1,52395.2


In [13]:
# check for erroneously repeated CI roots
tol = 1.e-6
for irrep, grp in dfci.sort_values('Energy').groupby(['Spin', 'Irrep']):
    e = grp.Energy.values
    de = e[1:] - e[:-1]
    smal = np.abs(de) < tol
    for i, s in enumerate(smal):
        if s:
            print('Warning: closely repeated root in MRCI')
            display(grp.iloc[[i,i+1]])
print('Checking for discrepancies between reference energy and CASSCF energy')
dfcheck = dfci[['Spin', 'Irrep', 'Label', 'Edav', 'Eref']].copy()
ecas = []
for i, row in dfcheck.iterrows():
    ecas.append(CAS.results[(CAS.results.Label == row.Label) & (CAS.results.Spin == row.Spin)].Energy.values[0])
dfcheck['CAS'] = ecas
dfcheck['diff'] = np.round(dfcheck.CAS - dfcheck.Eref, 6)
dfbad = dfcheck[np.abs(dfcheck['diff']) > 0.1].sort_values(['Spin', 'Irrep', 'Label'])
if len(dfbad):
    display(dfbad)
else:
    print('\t--looks good')

Checking for discrepancies between reference energy and CASSCF energy
	--looks good


In [14]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)
if SOCI.PG != 'C2v':
    print('*** Warning *** this script is only for C2v point group')
dfterms = SOCI.average_terms()

Computational group = C2v
CASSCF states:
    22 Doublet
Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "44 SO states but only 36 unique basis states" in parse_SOmatrix()


In [15]:
# Assign leading terms
lTerm = []  # list of str
for ilead in np.argmax(SOCI.termwt, axis=0):
    lTerm.append(SOCI.dfterm.at[ilead, 'Term'])
dfso = SOCI.SOe.results.copy()
dfso['Leading'] = lTerm
# All term weights
twts = []   # list of dict
for iso in range(SOCI.dimen):
    twts.append({t: np.round(w, 5) for t, w in zip(SOCI.dfterm.Term.values, SOCI.termwt[:, iso])})
dfso['Termwts'] = twts
print('Term weights rounded to 1e-5')
dfso

Term weights rounded to 1e-5


,Nr,Irrep,E,Eshift,Erel,Leading,Termwts
0,1,0,-661.189129,-0.56,0.00,(1)2Σ+,"{'(1)2Σ+': 0.99998, '(1)2Π': 0.0, '(2)2Π': 1e-..."
1,2,0,-661.189129,-0.56,0.00,(1)2Σ+,"{'(1)2Σ+': 0.99998, '(1)2Π': 0.0, '(2)2Π': 1e-..."
2,3,0,-661.066348,26946.58,26947.15,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99994, '(2)2Π': 3e-..."
3,4,0,-661.066348,26946.58,26947.15,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99994, '(2)2Π': 3e-..."
4,5,0,-661.066160,26987.84,26988.41,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99996, '(2)2Π': 3e-..."
5,6,0,-661.066160,26987.84,26988.41,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99996, '(2)2Π': 3e-..."
6,7,0,-661.026743,35638.98,35639.54,(2)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 3e-05, '(2)2Π': 0.999..."
7,8,0,-661.026743,35638.98,35639.54,(2)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 3e-05, '(2)2Π': 0.999..."
8,10,0,-661.024457,36140.67,36141.23,(2)2Π,"{'(1)2Σ+': 1e-05, '(1)2Π': 2e-05, '(2)2Π': 0.9..."
9,9,0,-661.024457,36140.67,36141.23,(2)2Π,"{'(1)2Σ+': 1e-05, '(1)2Π': 2e-05, '(2)2Π': 0.9..."


In [16]:
# Create a DataFrame to describe the basis states
cilbl = []
irrep = []
Ss = []
Szs = []
for bas in SOCI.basis:
    cilbl.append(bas[0])
    i, irp = bas[0].split('.')
    irrep.append(irp)
    Ss.append(bas[1])
    Szs.append(bas[2])
dfbas = pd.DataFrame({'cilbl': cilbl, 'irrep': irrep, 'S': Ss, 'Sz': Szs})

In [17]:
# Add values of |Lz|
Lz = [dfci.loc[ici, 'Lz'] for ici in SOCI.sob_ici]
dfbas['|Lz|'] = Lz

In [18]:
# Pair the basis states that are sin/cos siblings
sib = [-1] * SOCI.dimen
trig = [dfci.at[ici, 'trig'] for ici in SOCI.sob_ici]
dfbas['trig'] = trig
baspairs = []
for iterm, trow in SOCI.dfterm.iterrows():
    ibass = SOCI.term_iso[iterm]
    for ibas in ibass:
        if sib[ibas] > -1:
            # already assigned
            continue
        for jbas in ibass:
            if (jbas != ibas) and (dfbas.at[ibas, 'S'] == dfbas.at[jbas, 'S']) and \
                            (dfbas.at[ibas, 'Sz'] == dfbas.at[jbas, 'Sz']):
                sib[ibas] = jbas
                sib[jbas] = ibas
                baspairs.append([ibas, jbas])
dfbas['sib'] = sib
print('Basis states')
dfbas

Basis states


,cilbl,irrep,S,Sz,|Lz|,trig,sib
0,1.1,1,0.5,0.5,0.0,1,-1
1,2.1,1,0.5,0.5,0.0,1,-1
2,3.1,1,0.5,0.5,0.0,1,-1
3,5.1,1,0.5,0.5,0.0,1,-1
4,7.1,1,0.5,0.5,0.0,1,-1
5,8.1,1,0.5,0.5,0.0,1,-1
6,1.1,1,0.5,-0.5,0.0,1,-1
7,2.1,1,0.5,-0.5,0.0,1,-1
8,3.1,1,0.5,-0.5,0.0,1,-1
9,5.1,1,0.5,-0.5,0.0,1,-1


In [19]:
# Assign Omega value to each eigenvector 
thresh = 0.001  # minimum coefficient to consider
print('Threshold = {:.5f}'.format(thresh))
print('Everything below is numbered from zero, but in Molpro is numbered from 1')
Omegas = [None] * SOCI.dimen
for ivec in range(SOCI.dimen):
    print(f'Eigenvector #{ivec}')
    vec = SOCI.SOvec[:, ivec]
    #print(np.round(vec, 6))
    omset = set()  # set of found Omega values--should end up with only one element
    usedBS = []  # list of basis states considered
    # First consider sibling basis states
    for baspair in baspairs:
        print('\tBSs:', baspair)
        # 'a' and 'b' are coeffs of cos() and sin()
        [i, j] = baspair
        usedBS.extend(baspair)
        aLz = dfbas.at[i, '|Lz|']  # same for basis-state[i] and BS[j]
        Sz = dfbas.at[i, 'Sz']  # same for i and j
        if dfbas.at[i, 'trig'] == 'cos':
            # cos, sin ordering
            [a, b] = [vec[i], vec[j]]
        else:
            # sin, cos ordering
            [a, b] = [vec[j], vec[i]]
        #print('coeffs:', vec[i], vec[j])
        alen = np.absolute(a)
        blen = np.absolute(b)
        delta = alen - blen
        if abs(delta) > thresh:
            print('** large difference in magnitudes = {:.8f}'.format(delta))
        if max(alen, blen) < thresh:
            # ignore small coefficients
            continue
        # divide by a to get standard form
        b /= a
        a = 1
        #print('a, b  :', a, b)
        dev = abs(np.imag(b)) - 1
        if abs(dev) > thresh:
            print('** imag(b) deviates from unity by: {:.8f}'.format(dev))
        if np.imag(b) > thresh:
            # a positive combination
            Lz = aLz
        elif np.imag(b) < -thresh:
            # a negative combination
            Lz = -aLz
        Omz = Lz + Sz
        print(f'\t\tLz = {Lz}, Sz = {Sz}, Omz = {Omz}')
        omset = omset.union([abs(Omz)])
    # Now consider the other (Sigma) basis states
    for ibas in range(SOCI.dimen):
        if ibas in usedBS:
            continue
        usedBS.append(ibas)
        if np.absolute(vec[ibas]) < thresh:
            # small; ignore
            continue
        # 
        print(f'\tBS {ibas}')
        aLz = dfbas.at[ibas, '|Lz|']  # same for basis-state[i] and BS[j]
        if abs(aLz) != 0:
            print(f'\t\tThis should be a Sigma component but |Lz| = {aLz}')
        Sz = dfbas.at[ibas, 'Sz']
        Omz = aLz + Sz
        print(f'\t\tLz = {aLz}, Sz = {Sz}, Omz = {Omz}')
        omset = omset.union([abs(Omz)])
        
    # finish up
    if len(omset) == 1:
        Omegas[ivec] = omset.pop()
    else:
        print('*** Wrong number of Omega values!', omset)
    #print()

Threshold = 0.00100
Everything below is numbered from zero, but in Molpro is numbered from 1
Eigenvector #0
	BSs: [16, 26]
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [21, 31]
	BSs: [17, 27]
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [22, 32]
	BSs: [12, 36]
	BSs: [14, 38]
	BSs: [18, 28]
	BSs: [23, 33]
	BSs: [19, 29]
	BSs: [24, 34]
	BSs: [37, 13]
	BSs: [39, 15]
	BSs: [20, 30]
	BSs: [25, 35]
	BS 6
		Lz = 0.0, Sz = -0.5, Omz = -0.5
Eigenvector #1
	BSs: [16, 26]
	BSs: [21, 31]
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [17, 27]
	BSs: [22, 32]
		Lz = 1.0, Sz = -0.5, Omz = 0.5
	BSs: [12, 36]
	BSs: [14, 38]
	BSs: [18, 28]
	BSs: [23, 33]
	BSs: [19, 29]
	BSs: [24, 34]
	BSs: [37, 13]
	BSs: [39, 15]
	BSs: [20, 30]
	BSs: [25, 35]
	BS 0
		Lz = 0.0, Sz = 0.5, Omz = 0.5
Eigenvector #2
	BSs: [16, 26]
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [21, 31]
	BSs: [17, 27]
		Lz = -1.0, Sz = 0.5, Omz = -0.5
	BSs: [22, 32]
	BSs: [12, 36]
	BSs: [14, 38]
	BSs: [18, 28]
	BSs: [23, 33]
	BSs: [19, 29]
	BSs: [24, 34]
	BSs: [37, 1

In [20]:
dfso[chem.OMEGA] = Omegas
dfso

,Nr,Irrep,E,Eshift,Erel,Leading,Termwts,Ω
0,1,0,-661.189129,-0.56,0.00,(1)2Σ+,"{'(1)2Σ+': 0.99998, '(1)2Π': 0.0, '(2)2Π': 1e-...",0.5
1,2,0,-661.189129,-0.56,0.00,(1)2Σ+,"{'(1)2Σ+': 0.99998, '(1)2Π': 0.0, '(2)2Π': 1e-...",0.5
2,3,0,-661.066348,26946.58,26947.15,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99994, '(2)2Π': 3e-...",0.5
3,4,0,-661.066348,26946.58,26947.15,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99994, '(2)2Π': 3e-...",0.5
4,5,0,-661.066160,26987.84,26988.41,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99996, '(2)2Π': 3e-...",1.5
5,6,0,-661.066160,26987.84,26988.41,(1)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 0.99996, '(2)2Π': 3e-...",1.5
6,7,0,-661.026743,35638.98,35639.54,(2)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 3e-05, '(2)2Π': 0.999...",1.5
7,8,0,-661.026743,35638.98,35639.54,(2)2Π,"{'(1)2Σ+': 0.0, '(1)2Π': 3e-05, '(2)2Π': 0.999...",1.5
8,10,0,-661.024457,36140.67,36141.23,(2)2Π,"{'(1)2Σ+': 1e-05, '(1)2Π': 2e-05, '(2)2Π': 0.9...",0.5
9,9,0,-661.024457,36140.67,36141.23,(2)2Π,"{'(1)2Σ+': 1e-05, '(1)2Π': 2e-05, '(2)2Π': 0.9...",0.5


In [21]:
dflevel = mpr.average_SO_levels(dfso, be_same=['Ω'], to_avg=['E', 'Eshift', 'Erel'])
print(f'Assignments for {fsoci}')
dflevel[['Nr', 'E', 'Eshift', 'Erel', 'Leading', 'Ω', 'g']]

Assignments for ../UMemphis/mgcl-equil-karl-block.pro


,Nr,E,Eshift,Erel,Leading,Ω,g
0,"[1, 2]",-661.189129,-0.56,0.00,(1)2Σ+,0.5,2
2,"[3, 4]",-661.066348,26946.58,26947.15,(1)2Π,0.5,2
4,"[5, 6]",-661.066160,26987.84,26988.41,(1)2Π,1.5,2
6,"[7, 8]",-661.026743,35638.98,35639.54,(2)2Π,1.5,2
9,"[9, 10]",-661.024457,36140.67,36141.23,(2)2Π,0.5,2
10,"[11, 12]",-661.015063,38202.51,38203.07,(2)2Σ+,0.5,2
12,"[13, 14]",-660.992606,43131.07,43131.63,(3)2Σ+,0.5,2
14,"[15, 16]",-660.953297,51758.40,51758.96,(1)2Δ,1.5,2
16,"[17, 18]",-660.952326,51971.66,51972.23,(1)2Δ,2.5,2
18,"[19, 20]",-660.950459,52381.42,52381.98,(1)2Σ-,0.5,2
